In [112]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.decomposition import PCA
from scipy import stats
import joblib
import os


In [113]:
# Create output directory for plots if it doesn't exist
os.makedirs("plots", exist_ok=True)

print("DATASET OVERVIEW")
# Load the dataset
df = pd.read_csv("data/anxiety level.csv")
print(f"Dataset shape: {df.shape[0]} samples, {df.shape[1]} features")
# Describe dataset features and types
target_col = 'Anxiety Level (1-10)'
X = df.drop(target_col, axis=1)
y = df[target_col]

# Identify numerical and categorical features
num_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_cols = X.select_dtypes(include=['object']).columns.tolist()

print("\nFeature Overview:")
print(f"Target variable: {target_col} - Anxiety level on scale 1-10")
print(f"Numerical features ({len(num_cols)}):")
for col in num_cols:
    print(f"  - {col}: range {X[col].min():.1f}-{X[col].max():.1f}, mean {X[col].mean():.2f}")
print(f"Categorical features ({len(cat_cols)}):")
for col in cat_cols:
    print(f"  - {col}: {X[col].nunique()} unique values")

# Split data FIRST (to avoid data leakage)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Training set: {X_train.shape[0]} samples, Test set: {X_test.shape[0]} samples")

DATASET OVERVIEW
Dataset shape: 11000 samples, 19 features

Feature Overview:
Target variable: Anxiety Level (1-10) - Anxiety level on scale 1-10
Numerical features (11):
  - Age: range 18.0-64.0, mean 40.41
  - Sleep Hours: range 0.2-12.5, mean 6.77
  - Physical Activity (hrs/week): range 0.0-11.2, mean 2.92
  - Caffeine Intake (mg/day): range 0.0-599.0, mean 313.29
  - Alcohol Consumption (drinks/week): range 0.0-19.0, mean 9.73
  - Stress Level (1-10): range 1.0-10.0, mean 5.81
  - Heart Rate (bpm): range 60.0-119.0, mean 90.99
  - Breathing Rate (breaths/min): range 12.0-29.0, mean 20.79
  - Sweating Level (1-5): range 1.0-5.0, mean 3.08
  - Therapy Sessions (per month): range 0.0-9.0, mean 4.68
  - Diet Quality (1-10): range 1.0-10.0, mean 5.21
Categorical features (7):
  - Gender: 3 unique values
  - Occupation: 13 unique values
  - Smoking: 2 unique values
  - Family History of Anxiety: 2 unique values
  - Dizziness: 2 unique values
  - Medication: 2 unique values
  - Recent Maj

In [114]:
print("\nEXPLORATORY DATA ANALYSIS")

# Check for missing values
missing_values = X.isnull().sum()
if missing_values.sum() > 0:
    print("Missing values per column:")
    print(missing_values[missing_values > 0])
else:
    print("No missing values found")

# Target variable analysis
print(f"\nTarget Variable (Anxiety Level) Analysis:")
print(f"Mean: {y.mean():.2f}, Median: {y.median():.2f}, Min: {y.min()}, Max: {y.max()}")
print(f"Distribution: {y.value_counts().sort_index().to_dict()}")



EXPLORATORY DATA ANALYSIS
No missing values found

Target Variable (Anxiety Level) Analysis:
Mean: 4.15, Median: 4.00, Min: 1.0, Max: 10.0
Distribution: {1.0: 806, 2.0: 1524, 3.0: 2278, 4.0: 2467, 5.0: 1816, 6.0: 821, 7.0: 246, 8.0: 348, 9.0: 364, 10.0: 330}


In [115]:
# Create visualization for target variable
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
sns.histplot(y, kde=True, bins=10)
plt.title('Distribution of Anxiety Level')
plt.subplot(1, 2, 2)
sns.boxplot(y=y)
plt.title('Boxplot of Anxiety Level')
plt.tight_layout()
plt.savefig('plots/anxiety_distribution.png')
plt.close()
print("Saved target variable distribution plot to 'plots/anxiety_distribution.png'")


Saved target variable distribution plot to 'plots/anxiety_distribution.png'


In [116]:
# Correlation analysis
correlations = []
for col in num_cols:
    corr = X_train[col].corr(y_train)
    correlations.append((col, corr))
correlations.sort(key=lambda x: abs(x[1]), reverse=True) # To get the heighst correlation first

# for col, corr in correlations:
#     print(f"{col}: {corr:.4f}")

print("\nTop 5 correlations with Anxiety Level:") 
for col, corr in correlations[:5]: # taking only top 5 correlations
    print(f"{col}: {corr:.4f}")




Top 5 correlations with Anxiety Level:
Stress Level (1-10): 0.6570
Sleep Hours: -0.4673
Caffeine Intake (mg/day): 0.2810
Therapy Sessions (per month): 0.2626
Physical Activity (hrs/week): -0.2259


In [117]:
# Visualize top correlations
plt.figure(figsize=(12, 5))
corr_df = pd.DataFrame(correlations, columns=['Feature', 'Correlation'])
sns.barplot(x='Correlation', y='Feature', data=corr_df.head(10)) # Plotting the top 10 correlations
plt.title('Top 10 Features Correlated with Anxiety Level')
plt.tight_layout()
plt.savefig('plots/top_correlations.png')
plt.close()
print("Saved correlation plot to 'plots/top_correlations.png'")

Saved correlation plot to 'plots/top_correlations.png'


In [118]:
# showing upper traingle of matrix look
np.triu(np.ones([4,4])) 

# making upper traingle of boolean matrix 
pd.DataFrame(np.triu(np.ones_like(corr_matrix, dtype=bool))) 

,0,1,2,3,4,5,6,7,8,9,10,11
0,True,True,True,True,True,True,True,True,True,True,True,True
1,False,True,True,True,True,True,True,True,True,True,True,True
2,False,False,True,True,True,True,True,True,True,True,True,True
3,False,False,False,True,True,True,True,True,True,True,True,True
4,False,False,False,False,True,True,True,True,True,True,True,True
5,False,False,False,False,False,True,True,True,True,True,True,True
6,False,False,False,False,False,False,True,True,True,True,True,True
7,False,False,False,False,False,False,False,True,True,True,True,True
8,False,False,False,False,False,False,False,False,True,True,True,True
9,False,False,False,False,False,False,False,False,False,True,True,True


In [119]:
# Create correlation heatmap
plt.figure(figsize=(14, 12))
corr_matrix = df[num_cols + [target_col]].corr()

mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
# mask is a boolean matrix with the same shape as corr_matrix
# with True values in the upper triangle and False values in the lower triangle
sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.2f', cmap='coolwarm', linewidths = 0.5) 
plt.title('Correlation Matrix')
plt.tight_layout()
plt.savefig('plots/correlation_heatmap.png')
plt.close()
print("Saved correlation heatmap to 'plots/correlation_heatmap.png'")


Saved correlation heatmap to 'plots/correlation_heatmap.png'


In [120]:
# Key insights from EDA
print("\nKey EDA Insights:")
print("# Most factors that increase anxiety")
for col, corr in correlations[:3]:
    direction = "increases" if corr > 0 else "decreases"
    print(f"   - {col} {direction} anxiety (correlation: {corr:.2f})")


Key EDA Insights:
# Most factors that increase anxiety
   - Stress Level (1-10) increases anxiety (correlation: 0.66)
   - Sleep Hours decreases anxiety (correlation: -0.47)
   - Caffeine Intake (mg/day) increases anxiety (correlation: 0.28)


In [121]:
print("Anxiety level distribution shape")
skewness = y.skew()
print(f"   - Skewness: {skewness:.2f} ({'positively skewed' if skewness > 0 else 'negatively skewed' if skewness < 0 else 'symmetric'})")


Anxiety level distribution shape
   - Skewness: 0.92 (positively skewed)


In [ ]:
print("\nDATA CLEANING")
# Handle outliers
def identify_outliers(series):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return series[(series < lower_bound) | (series > upper_bound)]

# Detect outliers in numerical features
outlier_counts = {}
for col in num_cols:
    outliers = identify_outliers(X_train[col])
    if len(outliers) > 0:
        outlier_counts[col] = len(outliers)
        print(f"{col}: {len(outliers)} outliers ({len(outliers)/len(X_train)*100:.1f}%)")

# Create preprocessing steps
# One-hot encode categorical features
X_train_encoded = pd.get_dummies(X_train, columns=cat_cols, drop_first=True)
X_test_encoded = pd.get_dummies(X_test, columns=cat_cols, drop_first=True)

# Ensure test set has same columns as train set
for col in X_train_encoded.columns:
    if col not in X_test_encoded.columns:
        X_test_encoded[col] = 0
X_test_encoded = X_test_encoded[X_train_encoded.columns]

# Scale numerical features
scaler = StandardScaler()
X_train_scaled = X_train_encoded.copy()
X_test_scaled = X_test_encoded.copy()
X_train_scaled[num_cols] = scaler.fit_transform(X_train_scaled[num_cols])
X_test_scaled[num_cols] = scaler.transform(X_test_scaled[num_cols])
print("Completed data preprocessing (encoding and scaling)")



DATA CLEANING
Sleep Hours: 24 outliers (0.3%)
Physical Activity (hrs/week): 19 outliers (0.2%)
Completed data preprocessing (encoding and scaling)
30


In [123]:
print("\nDIMENSIONALITY REDUCTION")
# Use PCA for numerical features only
print("Applying Principal Component Analysis (PCA):")
pca = PCA(n_components=0.95)  # Retain 95% variance
X_train_num_pca = pca.fit_transform(X_train_scaled[num_cols])
X_test_num_pca = pca.transform(X_test_scaled[num_cols])

print(f"PCA reduced dimensions from {len(num_cols)} to {pca.n_components_} components")
print(f"Explained variance: {np.sum(pca.explained_variance_ratio_):.2f}")
print(f"Component contribution:")
for i, var in enumerate(pca.explained_variance_ratio_):
    print(f"  - PC{i+1}: {var:.4f} ({var*100:.1f}% of variance)")

# Visualize explained variance
plt.figure(figsize=(8, 4))
plt.plot(np.cumsum(pca.explained_variance_ratio_), marker='o')
plt.axhline(y=0.95, color='r', linestyle='--')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('Explained Variance by PCA Components')
plt.savefig('plots/pca_variance.png')
plt.close()
print("Saved PCA variance plot to 'plots/pca_variance.png'")



DIMENSIONALITY REDUCTION
Applying Principal Component Analysis (PCA):
PCA reduced dimensions from 11 to 11 components
Explained variance: 1.00
Component contribution:
  - PC1: 0.1523 (15.2% of variance)
  - PC2: 0.0929 (9.3% of variance)
  - PC3: 0.0897 (9.0% of variance)
  - PC4: 0.0875 (8.7% of variance)
  - PC5: 0.0868 (8.7% of variance)
  - PC6: 0.0862 (8.6% of variance)
  - PC7: 0.0847 (8.5% of variance)
  - PC8: 0.0835 (8.4% of variance)
  - PC9: 0.0823 (8.2% of variance)
  - PC10: 0.0791 (7.9% of variance)
  - PC11: 0.0750 (7.5% of variance)
Saved PCA variance plot to 'plots/pca_variance.png'


In [124]:
print("\nMODELING PIPELINE")
print("Creating modeling pipeline with preprocessing and model training steps:")
print("1. One-hot encoding of categorical features")
print("2. Standardization of numerical features")
print("3. Linear regression model fitting")
print("4. Model evaluation")

# Function to evaluate models
def evaluate_model(name, model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    print(f"{name}: R² = {r2:.4f}, RMSE = {rmse:.4f}, MAE = {mae:.4f}")
    return model, y_pred, r2, rmse, mae


MODELING PIPELINE
Creating modeling pipeline with preprocessing and model training steps:
1. One-hot encoding of categorical features
2. Standardization of numerical features
3. Linear regression model fitting
4. Model evaluation


In [125]:
print("\nHYPERPARAMETER TUNING AND EVALUATION")

# Base Linear Regression (no hyperparameters to tune)
print("\nBaseline Linear Regression:")
lr_model, lr_pred, lr_r2, lr_rmse, lr_mae = evaluate_model(
    "Linear Regression", LinearRegression(), 
    X_train_scaled, X_test_scaled, y_train, y_test
)

# Ridge Regression with hyperparameter tuning
print("\nRidge Regression with hyperparameter tuning:")
ridge_alphas = [0.01, 0.1, 1.0, 10.0, 100.0]
best_ridge_r2 = -np.inf
best_ridge_alpha = None
best_ridge_model = None

for alpha in ridge_alphas:
    model, pred, r2, rmse, mae = evaluate_model(
        f"Ridge (alpha={alpha})", Ridge(alpha=alpha),
        X_train_scaled, X_test_scaled, y_train, y_test
    )
    if r2 > best_ridge_r2:
        best_ridge_r2 = r2
        best_ridge_alpha = alpha
        best_ridge_model = model
        best_ridge_pred = pred
        best_ridge_rmse = rmse
        best_ridge_mae = mae

print(f"Best Ridge model: alpha={best_ridge_alpha}, R² = {best_ridge_r2:.4f}")

# Lasso Regression with hyperparameter tuning
print("\nLasso Regression with hyperparameter tuning:")
lasso_alphas = [0.001, 0.01, 0.1, 1.0, 10.0]
best_lasso_r2 = -np.inf
best_lasso_alpha = None
best_lasso_model = None

for alpha in lasso_alphas:
    model, pred, r2, rmse, mae = evaluate_model(
        f"Lasso (alpha={alpha})", Lasso(alpha=alpha, max_iter=10000),
        X_train_scaled, X_test_scaled, y_train, y_test
    )
    if r2 > best_lasso_r2:
        best_lasso_r2 = r2
        best_lasso_alpha = alpha
        best_lasso_model = model
        best_lasso_pred = pred
        best_lasso_rmse = rmse
        best_lasso_mae = mae

print(f"Best Lasso model: alpha={best_lasso_alpha}, R² = {best_lasso_r2:.4f}")

# Compare models and select the best
models = {
    "Linear Regression": (lr_model, lr_pred, lr_r2, lr_rmse),
    "Ridge": (best_ridge_model, best_ridge_pred, best_ridge_r2, best_ridge_rmse),
    "Lasso": (best_lasso_model, best_lasso_pred, best_lasso_r2, best_lasso_rmse)
}

best_model_name = max(models.keys(), key=lambda k: models[k][2])
best_model, best_pred, best_r2, best_rmse = models[best_model_name]

print(f"\nBest model: {best_model_name} (R² = {best_r2:.4f}, RMSE = {best_rmse:.4f})")

# Visualize actual vs predicted values
plt.figure(figsize=(8, 6))
plt.scatter(y_test, best_pred, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.title(f'{best_model_name}: Actual vs Predicted Anxiety Levels')
plt.xlabel('Actual Anxiety Level')
plt.ylabel('Predicted Anxiety Level')
plt.savefig('plots/actual_vs_predicted.png')
plt.close()
print("Saved actual vs predicted plot to 'plots/actual_vs_predicted.png'")


HYPERPARAMETER TUNING AND EVALUATION

Baseline Linear Regression:
Linear Regression: R² = 0.6773, RMSE = 1.2172, MAE = 0.9643

Ridge Regression with hyperparameter tuning:
Ridge (alpha=0.01): R² = 0.6773, RMSE = 1.2172, MAE = 0.9643
Ridge (alpha=0.1): R² = 0.6773, RMSE = 1.2172, MAE = 0.9643
Ridge (alpha=1.0): R² = 0.6773, RMSE = 1.2172, MAE = 0.9643
Ridge (alpha=10.0): R² = 0.6773, RMSE = 1.2172, MAE = 0.9643
Ridge (alpha=100.0): R² = 0.6772, RMSE = 1.2173, MAE = 0.9642
Best Ridge model: alpha=0.01, R² = 0.6773

Lasso Regression with hyperparameter tuning:
Lasso (alpha=0.001): R² = 0.6773, RMSE = 1.2172, MAE = 0.9641
Lasso (alpha=0.01): R² = 0.6764, RMSE = 1.2187, MAE = 0.9652
Lasso (alpha=0.1): R² = 0.6568, RMSE = 1.2552, MAE = 0.9839
Lasso (alpha=1.0): R² = 0.2057, RMSE = 1.9096, MAE = 1.4225
Lasso (alpha=10.0): R² = -0.0002, RMSE = 2.1428, MAE = 1.6146
Best Lasso model: alpha=0.001, R² = 0.6773

Best model: Linear Regression (R² = 0.6773, RMSE = 1.2172)
Saved actual vs predicted p

In [126]:
print("\nREGRESSION DIAGNOSTICS")
# Calculate residuals
residuals = y_test - best_pred

# Normality test for residuals
stat, p = stats.shapiro(residuals)
print(f"Shapiro-Wilk test for normality: p-value = {p:.4f}")
print("Residuals are normally distributed" if p > 0.05 else "Residuals are NOT normally distributed")

# Create diagnostic plots
plt.figure(figsize=(12, 10))
# Residuals vs Fitted
plt.subplot(2, 2, 1)
plt.scatter(best_pred, residuals, alpha=0.5)
plt.axhline(y=0, color='r', linestyle='--')
plt.title('Residuals vs Fitted Values')
plt.xlabel('Fitted Values')
plt.ylabel('Residuals')

# QQ plot
plt.subplot(2, 2, 2)
stats.probplot(residuals, plot=plt)
plt.title('Normal Q-Q Plot')

# Scale-Location plot
plt.subplot(2, 2, 3)
plt.scatter(best_pred, np.abs(residuals), alpha=0.5)
plt.title('Scale-Location Plot')
plt.xlabel('Fitted Values')
plt.ylabel('|Residuals|')

# Residuals histogram
plt.subplot(2, 2, 4)
sns.histplot(residuals, kde=True)
plt.title('Distribution of Residuals')
plt.tight_layout()
plt.savefig('plots/regression_diagnostics.png')
plt.close()
print("Saved regression diagnostic plots to 'plots/regression_diagnostics.png'")

# Save the best model for deployment
joblib.dump(best_model, 'models/anxiety_model.pkl')
# Save scaler and columns for deployment
joblib.dump(scaler, 'models/scaler.pkl')
joblib.dump({'num_cols': num_cols, 'cat_cols': cat_cols, 'model_columns': X_train_scaled.columns.tolist()}, 'models/columns.pkl')


REGRESSION DIAGNOSTICS
Shapiro-Wilk test for normality: p-value = 0.0039
Residuals are NOT normally distributed
Saved regression diagnostic plots to 'plots/regression_diagnostics.png'


['models/columns.pkl']